# Implement A Data Cleaning Agent

Business Science University

AI Data Science Team

### Load Libraries

In [1]:
# * Libraries

from langchain_openai import ChatOpenAI
import os
import yaml
import pandas as pd
from pprint import pprint

from ai_data_science_team.agents import data_cleaning_agent

### Setup AI and Logging

This section of code sets up the LLM inputs and the logging information. Logging is used to store AI-generated code and files during the AI Data Science Teams processing of files. 

*Important Note:* This example uses OpenAI's API. But any LLM can be used such as Anthropic or local LLMs with Ollama.

In [4]:
# * Setup

MODEL    = "gpt-4o-mini"
LOG      = True
LOG_PATH = os.path.join(os.getcwd(), "logs/")

os.environ["OPENAI_API_KEY"] = yaml.safe_load(open('../credentials.yml'))['openai']

llm = ChatOpenAI(model = MODEL)

llm


ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7fe9c8cf03d0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7fe9b8806fb0>, root_client=<openai.OpenAI object at 0x7fe9d8db2920>, root_async_client=<openai.AsyncOpenAI object at 0x7fe9c8cf21d0>, model_name='gpt-4o-mini', model_kwargs={}, openai_api_key=SecretStr('**********'))

### Load a Dataset

Next, let's load a customer churn data set that we will clean up. 

In [5]:
df = pd.read_csv("data/churn_data.csv")
df

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,6840-RESVB,Male,0,Yes,Yes,24,Yes,Yes,DSL,Yes,...,Yes,Yes,Yes,Yes,One year,Yes,Mailed check,84.80,1990.5,No
7039,2234-XADUH,Female,0,Yes,Yes,72,Yes,Yes,Fiber optic,No,...,Yes,No,Yes,Yes,One year,Yes,Credit card (automatic),103.20,7362.9,No
7040,4801-JZAZL,Female,0,Yes,Yes,11,No,No phone service,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.60,346.45,No
7041,8361-LTMKD,Male,1,Yes,No,4,Yes,Yes,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Mailed check,74.40,306.6,Yes


### Create The Agent

Run this code to create the agent. The main inputs to the `data_cleaning_agent()` are:

- **user_instructions**: The data cleaning agent will use these comments to modify the "standard recipe" 
  - Standard Recipe: The standard cleaning recipe which includes removing columns with more than 40% missing values, imputing missing values using mean (numeric) or mode (categorical), removing duplicate rows, and removing outliers. 
- **data_raw**: The raw data to be cleaned
- **max_retries**: Used to limit the number of attempts to fix the python code generated by the agent. Set this to 3 to limit to 3 attempts. 
- **retry_count**: Set this to 0. 

In [7]:
data_cleaning_agent = data_cleaning_agent(model = llm, log=LOG, log_path=LOG_PATH)

response = data_cleaning_agent.invoke({
    "user_instructions": "Don't remove outliers when cleaning the data.",
    "data_raw": df.to_dict(),
    "max_retries":3, 
    "retry_count":0
})

---DATA CLEANING AGENT----
    * CREATE DATA CLEANER CODE
    * EXECUTING AGENT CODE
    * EXPLAIN AGENT CODE


### Response

The response produced contains everything we need to understand the data cleaning decisions made and get the cleaned dataset. 

In [10]:
list(response.keys())

['messages',
 'user_instructions',
 'data_raw',
 'data_cleaner_function',
 'data_cleaner_error',
 'data_cleaned',
 'max_retries',
 'retry_count']

### The cleaning recipe:

In [11]:
pprint(response['messages'][0].content)


('# Data Cleaning Agent:\n'
 '\n'
 ' The data cleaning agent performs the following steps in the `data_cleaner` '
 'function:\n'
 '\n'
 '1. **Remove Columns with Excessive Missing Values**: Columns with more than '
 '40% missing values are dropped from the dataset.\n'
 '\n'
 '2. **Impute Missing Values**:\n'
 '   - **Numeric Columns**: Missing values in numeric columns are filled using '
 'mean imputation.\n'
 '   - **Categorical Columns**: Missing values in categorical columns are '
 'filled using mode imputation.\n'
 '\n'
 "3. **Convert Data Types**: The 'TotalCharges' column is converted to a "
 'numeric type, with non-convertible entries set to NaN.\n'
 '\n'
 '4. **Remove Duplicate Rows**: Any duplicate rows in the dataset are '
 'removed.\n'
 '\n'
 '5. **Remove Rows with Missing Values**: Any remaining rows containing '
 'missing values are eliminated from the dataset.\n'
 '\n'
 'Note: The function intentionally avoids handling outliers.')


#### Data Cleaner Function

In [12]:
pprint(response['data_cleaner_function'])

('def data_cleaner(data_raw):\n'
 '    import pandas as pd\n'
 '    import numpy as np\n'
 '    from sklearn.impute import SimpleImputer\n'
 '\n'
 '    # Step 1: Remove columns with more than 40% missing values\n'
 '    threshold = 0.4 * len(data_raw)\n'
 '    data_cleaned = data_raw.loc[:, data_raw.isnull().sum() <= threshold]\n'
 '\n'
 '    # Step 2: Impute missing values\n'
 '    # For numeric columns, use mean imputation\n'
 '    numeric_cols = data_cleaned.select_dtypes(include=[np.number]).columns\n'
 "    imputer_numeric = SimpleImputer(strategy='mean')\n"
 '    data_cleaned[numeric_cols] = '
 'imputer_numeric.fit_transform(data_cleaned[numeric_cols])\n'
 '\n'
 '    # For categorical columns, use mode imputation\n'
 '    categorical_cols = data_cleaned.select_dtypes(include=[object]).columns\n'
 "    imputer_categorical = SimpleImputer(strategy='most_frequent')\n"
 '    data_cleaned[categorical_cols] = '
 'imputer_categorical.fit_transform(data_cleaned[categorical_cols])\n'
 '\n

#### Cleaned Data As Pandas Data Frame

In [13]:
pd.DataFrame(response['data_cleaned'])


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0.0,Yes,No,1.0,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0.0,No,No,34.0,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,3668-QPYBK,Male,0.0,No,No,2.0,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0.0,No,No,45.0,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0.0,No,No,2.0,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,6840-RESVB,Male,0.0,Yes,Yes,24.0,Yes,Yes,DSL,Yes,...,Yes,Yes,Yes,Yes,One year,Yes,Mailed check,84.80,1990.50,No
7039,2234-XADUH,Female,0.0,Yes,Yes,72.0,Yes,Yes,Fiber optic,No,...,Yes,No,Yes,Yes,One year,Yes,Credit card (automatic),103.20,7362.90,No
7040,4801-JZAZL,Female,0.0,Yes,Yes,11.0,No,No phone service,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.60,346.45,No
7041,8361-LTMKD,Male,1.0,Yes,No,4.0,Yes,Yes,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Mailed check,74.40,306.60,Yes
